<a href="https://colab.research.google.com/github/anumadhyani/mywork/blob/master/Detect_image_real_or_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests

In [2]:
!pip install pillow

In [30]:
import os
from PIL import Image
import requests
from io import BytesIO
import hashlib # Added for generating unique filenames
import cv2
import numpy as np
from scipy.fftpack import fft2, fftshift
from skimage.restoration import denoise_tv_chambolle

from google.colab import userdata
unsplashkey = userdata.get('Unsplash_Key')

from google.colab import drive
drive.mount('/content/drive') # Removed force_remount=True

# Define the path where you want to save your images in Google Drive
GDRIVE_SAVE_PATH = '/content/drive/MyDrive/Unsplash_Images'
print(f"Images will be saved to: {GDRIVE_SAVE_PATH}")

# Ensure the directory exists
os.makedirs(GDRIVE_SAVE_PATH, exist_ok=True)

print("a")

def fetch_image_urls(query):

  api_url =  f"https://api.unsplash.com/search/photos/?query={query}&client_id={unsplashkey}"
  responses = requests.get(api_url)

  if responses.status_code != 200:
    print("Error fetching image data")
    return

  data = responses.json()
  print("hi")
  urls = [result["urls"]["regular"] for result in data["results"]]
  print("here")
  return urls

def download_and_save_images(image_url, base_save_path): # Renamed parameter for clarity
  print(f"Attempting to download: {image_url}") # Clarified print statement
  response = requests.get(image_url)
  if response.status_code == 200:
    image = Image.open(BytesIO(response.content))
    #Determine file extension from content type or default to .jpg
    content_type = response.headers.get('content-type', 'image/jpeg')
    extension = '.' + content_type.split('/')[-1]

    # Handle cases where content type might not provide a standard extension, or add more common types
    if extension not in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']:
      extension = '.jpg' # Fallback extension

    # Generate a unique filename using a hash of the URL
    filename = hashlib.md5(image_url.encode('utf-8')).hexdigest() + extension
    full_save_path = os.path.join(base_save_path, filename)

    print(f"Saving image to: {full_save_path}")
    image.save(full_save_path)
    print(full_save_path)
    #extract_all_features(full_save_path)
    print(f"Successfully saved to {full_save_path}")
  else:
    print(f"Failed to download image from {image_url}. Status code: {response.status_code}")

def extract_noise_features(image_path):
    """
    Extract noise and frequency features from an image
    """
    # Load image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    features = {}

    # 1. NOISE RESIDUAL EXTRACTION
    # Denoise the image
    denoised = denoise_tv_chambolle(gray, weight=0.1)
    # Extract noise by subtracting denoised from original
    noise_residual = gray - denoised

    # Noise statistics
    features['noise_mean'] = np.mean(noise_residual)
    features['noise_std'] = np.std(noise_residual)
    features['noise_variance'] = np.var(noise_residual)

    # 2. FREQUENCY DOMAIN ANALYSIS
    # Apply FFT
    fft = fft2(gray)
    fft_shift = fftshift(fft)
    magnitude_spectrum = np.abs(fft_shift)

    # Divide into frequency bands
    h, w = magnitude_spectrum.shape
    center_h, center_w = h // 2, w // 2

    # Low frequency (center 25%)
    low_freq = magnitude_spectrum[
        center_h - h//8:center_h + h//8,
        center_w - w//8:center_w + w//8
    ]
    features['low_freq_energy'] = np.mean(low_freq)

    # High frequency (outer edges)
    high_freq_mask = np.ones_like(magnitude_spectrum)
    high_freq_mask[
        center_h - h//4:center_h + h//4,
        center_w - w//4:center_w + w//4
    ] = 0
    high_freq = magnitude_spectrum * high_freq_mask
    features['high_freq_energy'] = np.mean(high_freq[high_freq > 0])

    # Frequency ratio (AI images often have unnatural frequency distribution)
    features['freq_ratio'] = features['high_freq_energy'] / (features['low_freq_energy'] + 1e-10)

    # 3. TEXTURE ANALYSIS
    # Compute gradients
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)

    features['gradient_mean'] = np.mean(gradient_magnitude)
    features['gradient_std'] = np.std(gradient_magnitude)

    # 4. LOCAL VARIANCE ANALYSIS
    # AI images often have unnatural smoothness in certain regions
    kernel_size = 5
    local_means = cv2.blur(gray.astype(float), (kernel_size, kernel_size))
    local_sq_means = cv2.blur((gray.astype(float))**2, (kernel_size, kernel_size))
    local_variance = local_sq_means - local_means**2

    features['local_var_mean'] = np.mean(local_variance)
    features['local_var_std'] = np.std(local_variance)

    # 5. EDGE COHERENCE
    # AI images may have inconsistent edges
    edges = cv2.Canny(gray, 100, 200)
    features['edge_density'] = np.sum(edges > 0) / edges.size

    return features

def extract_all_features(image_path):
    """
    Wrapper to extract all features and return as array
    """
    features = extract_noise_features(image_path)
    # Return as ordered array for ML model
    return np.array([
        features['noise_mean'],
        features['noise_std'],
        features['noise_variance'],
        features['low_freq_energy'],
        features['high_freq_energy'],
        features['freq_ratio'],
        features['gradient_mean'],
        features['gradient_std'],
        features['local_var_mean'],
        features['local_var_std'],
        features['edge_density']
    ])

urls = fetch_image_urls("coding on laptop")
if urls:
  print(f"Found {len(urls)} image URLs.")
  for url in urls:
    download_and_save_images(url, GDRIVE_SAVE_PATH)


Mounted at /content/drive
Images will be saved to: /content/drive/MyDrive/Unsplash_Images
a
hi
here
Found 10 image URLs.
Attempting to download: https://images.unsplash.com/photo-1659642136446-41ab12578991?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w4NTgxMjh8MHwxfHNlYXJjaHwxfHxjb2RpbmclMjBvbiUyMGxhcHRvcHxlbnwwfHx8fDE3Njg1NTM1NDF8MA&ixlib=rb-4.1.0&q=80&w=1080
Saving image to: /content/drive/MyDrive/Unsplash_Images/6f23b50b9a91d8e3cc3b273eb3d7be31.jpeg
Successfully saved to /content/drive/MyDrive/Unsplash_Images/6f23b50b9a91d8e3cc3b273eb3d7be31.jpeg
Attempting to download: https://images.unsplash.com/photo-1569693799105-4eb645d89aea?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w4NTgxMjh8MHwxfHNlYXJjaHwyfHxjb2RpbmclMjBvbiUyMGxhcHRvcHxlbnwwfHx8fDE3Njg1NTM1NDF8MA&ixlib=rb-4.1.0&q=80&w=1080
Saving image to: /content/drive/MyDrive/Unsplash_Images/7d30edb29dbfbc8dec8475b87338a5c2.jpeg
Successfully saved to /content/drive/MyDrive/Unsplash_Images/7d30edb29dbfbc8dec8475b87338a5c2.jpeg
Attem

First, we need to mount your Google Drive to be able to save files there. You will be prompted to authorize this connection.

Mounted at /content/drive
Images will be saved to: /content/drive/MyDrive/Unsplash_Images
